# Full Model Notebook

## OUTSTANDING WORK
* Sync train-test split
* Incorporate ALMA text preprocessing and compare performance to our preprocessing
* Ensure measurements with width greater than 5GHz are dropped (I think this is only 2)
* Band EDA
* Remove outlier projects (> 26.5 measurement) from Band prediction
* Remove projects that have incorrectly formatted band data
    * E.G. 2011.0.00008.E has an observation line with band = '3 6'
* Test different text preprocessing options and compare results
* Consider removing bands 1 AND 2 from band prediction
    * Only 21 measurements in band 1, no measurements in band 2

## Workflow Outline:
We leverage two parallel pipelines, that are combined to recommend median frequencies to explore after each model has completed training and prediction.

All projects for this phase of the overall pipeline are 'line' projects.

### Frequency Mining Pipeline
* OPTIONAL: remove projects with > 26.5 measurements **CURRENTLY REMOVING**
    * Tested both options, hit rate accuracies did not increase significantly to offset 1k cluster add

* Run projects through LDA to generate topic model with $N=50$ topics
    * Currently using count vectorization of combined title and abstract with lemmatized_no_sw_text
* Group projects to max topic by taking argmax of document-topic table
* Run HDBSCAN on each of the topics to create measurement clusters, referred to as "areas of interest"
    * Currently areas of interest are taken from min and max median frequency for each cluster generated
    * NOTE: each of the 50 HDBSCAN models can (and probably should) be tuned individually
        * We should make sure generated clusters are not too large unless it makes sense
            * E.G. a large cluster from 700-750GHz might make sense since measurements in this range are generally sparse
            * These large clusters are due to HDBSCAN adjusting the "neighborhood size", $\epsilon$ dynamically (using heirarchical clustering underneath the hood) to account for areas of varying density, as opposed to DBSCAN which uses a flat $\epsilon$ for all measurements within a topic.

### Band Prediction Pipeline
* OPTIONAL: remove projects with > 26.5 measurements **NOT CURRENTLY REMOVING NEED TO CHANGE**
* Predict band for project with Naive Bayes
    * Currently using TF-IDF vectorization of combined title and abstract with **NEED TO CHOOSE TEXT**
* Choose band(s) using hard classification into one or two bands
    * We remove band 2 entirely because there are so few 
    * We do this to be able to give a final hit rate of appx. 75%
        * This shows we have a good prediction model to match projects to band
* Ultimately we will use probability vector output (not hard classification) to order mined recommendations by full band prediction

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.figure_factory as ff
from ast import literal_eval
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import DBSCAN, HDBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

SEED = 42

## Read data

In [2]:
train_projects = pd.read_csv("../data/train_projects.csv")
train_projects = train_projects.set_index('project_code')
train_projects.shape

(2383, 12)

In [3]:
test_projects = pd.read_csv("../data/test_projects.csv")
test_projects = test_projects.set_index('project_code')
test_projects.shape

(795, 12)

In [4]:
train_measurements = pd.read_csv('../../train_measurements.csv')
train_measurements = train_measurements.set_index('project_code')

In [5]:
test_measurements = pd.read_csv('../../test_measurements.csv')
test_measurements = test_measurements.set_index('project_code')

## Read in band predictions
This data frame gives a list from least likely band to most likely band for each test project

In [6]:
band_predictions = pd.read_csv('../data/diverse_band_prediction.csv')
band_predictions = band_predictions.set_index('project_code')
band_predictions.head()

,band_predictions
project_code,
2016.1.00485.S,"[1, 10, 9, 5, 6, 7, 4, 3, 8]"
2017.1.00824.S,"[1, 10, 5, 9, 8, 3, 6, 4, 7]"
2015.1.01088.S,"[1, 10, 9, 6, 5, 8, 4, 7, 3]"
2013.1.00781.S,"[1, 10, 9, 5, 8, 7, 4, 3, 6]"
2016.1.00800.S,"[1, 10, 5, 9, 4, 8, 3, 6, 7]"


## By hand band lower-bound cutoffs
To avoid possible conflicts, we simply call the cutoffs for band 1 and 2 to be 0 and 1GHz, respectively.

In [7]:
# band_cutoffs = [0, 1, 84, 120, 163, 211, 275, 385, 602, 787] old
band_cutoffs = [35, 51, 84, 125, 158, 211, 275, 385, 602, 787]

In [8]:
train_texts = train_projects.lemmatized_no_sw_text
test_texts = test_projects.lemmatized_no_sw_text

### LDA class

In [9]:
class LDA_Model:
    def __init__(self, N_topics=50):
        self.N_topics = N_topics
        self.countVectorizer = CountVectorizer()
        self.lda = LatentDirichletAllocation(n_components=self.N_topics, random_state=SEED)
    
    def fit(self, corpus):
        termFrequency = self.countVectorizer.fit_transform(corpus)
        self.lda.fit(termFrequency)
        return self.lda.transform(termFrequency)

    # Additional method to transform new data
    def transform(self, corpus):
        termFrequency = self.countVectorizer.transform(corpus)
        return self.lda.transform(termFrequency)

#### Initialize Model

In [10]:
lda_model = LDA_Model(N_topics=50)

#### Fit model on training set

In [11]:
train_topics = lda_model.fit(train_texts)

In [12]:
words = lda_model.countVectorizer.get_feature_names_out()

In [13]:
N = 10 #number of top words to show
topic_components = lda_model.lda.components_

for topic_idx, topic in enumerate(topic_components):
    print(f"Topic {topic_idx}:")
    # Get the indices of the top N words for this topic
    top_word_indices = topic.argsort()[-N:][::-1]
    # Print these words with their weights
    for word_idx in top_word_indices:
        print(f"{words[word_idx]} (weight: {topic[word_idx]:.2f})")
    print("\n")

Topic 0:
line (weight: 57.87)
bd (weight: 48.98)
compact (weight: 36.90)
velocity (weight: 26.63)
nucleus (weight: 24.94)
con (weight: 21.02)
obscure (weight: 17.56)
bds (weight: 17.02)
variation (weight: 16.27)
nuclei (weight: 15.03)


Topic 1:
dust (weight: 188.60)
gas (weight: 122.90)
ci (weight: 59.72)
grain (weight: 40.69)
evolution (weight: 39.17)
observation (weight: 36.21)
star (weight: 30.49)
carbon (weight: 30.10)
destruction (weight: 28.51)
study (weight: 26.06)


Topic 2:
agb (weight: 141.51)
star (weight: 127.80)
mass (weight: 107.10)
loss (weight: 81.86)
bipolar (weight: 70.32)
outflow (weight: 61.83)
jet (weight: 59.40)
nebula (weight: 53.18)
planetary (weight: 40.83)
wind (weight: 40.55)


Topic 3:
system (weight: 74.58)
debris (weight: 63.54)
belt (weight: 47.11)
planet (weight: 46.05)
disk (weight: 36.87)
collision (weight: 34.83)
structure (weight: 34.27)
disc (weight: 30.83)
observation (weight: 30.48)
scale (weight: 28.42)


Topic 4:
galaxy (weight: 212.35)
scale (

In [14]:
train_doc_topic = pd.DataFrame(train_topics)
train_doc_topic = train_doc_topic.set_index(train_texts.index.values)
train_doc_topic.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
2016.1.01288.S,0.000392,0.000392,0.000392,0.000392,0.000392,0.065492,0.000392,0.000392,0.000392,0.000392,...,0.000392,0.000392,0.000392,0.000392,0.000392,0.000392,0.464354,0.000392,0.000392,0.000392
2018.1.01077.S,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,...,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.376111,0.000194,0.000194
2018.1.00437.S,0.000192,0.000192,0.000192,0.127172,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,...,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192
2021.1.00637.S,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,...,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222
2012.1.00786.S,0.000171,0.000171,0.192147,0.000171,0.000171,0.000171,0.000171,0.740628,0.000171,0.000171,...,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171


In [15]:
train_texts = pd.DataFrame(train_texts)

### Match test data into topics

In [16]:
test_topics = lda_model.transform(test_texts)

In [17]:
test_doc_topic= pd.DataFrame(test_topics.tolist())
test_doc_topic= test_doc_topic.set_index(test_texts.index.values)
test_doc_topic.head(5)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
2016.1.00485.S,0.000215,0.000215,0.000215,0.029035,0.000215,0.000215,0.000215,0.000215,0.000215,0.000215,...,0.000215,0.000215,0.000215,0.000215,0.000215,0.000215,0.000215,0.031655,0.000215,0.113737
2017.1.00824.S,0.000169,0.098192,0.080003,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,...,0.000169,0.130644,0.000169,0.140330,0.000169,0.000169,0.000169,0.022981,0.000169,0.000169
2015.1.01088.S,0.188665,0.000211,0.000211,0.000211,0.017753,0.134495,0.000211,0.063180,0.063875,0.000211,...,0.000211,0.000211,0.000211,0.109437,0.000211,0.020642,0.146402,0.000211,0.000211,0.070094
2013.1.00781.S,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,0.135778,0.000294,0.000294,0.122034,...,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,0.000294,0.087903,0.000294,0.461476
2016.1.00800.S,0.317005,0.000182,0.000182,0.000182,0.000182,0.000182,0.000182,0.000182,0.000182,0.000182,...,0.000182,0.000182,0.000182,0.076003,0.000182,0.000182,0.000182,0.000182,0.052253,0.000182


In [18]:
test_texts = pd.DataFrame(test_texts)

### Group documents to highest matching topic

Combine project topic vector frames

In [19]:
proj_topics = pd.concat([train_doc_topic, test_doc_topic])
proj_topics.head(5)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
2016.1.01288.S,0.000392,0.000392,0.000392,0.000392,0.000392,0.065492,0.000392,0.000392,0.000392,0.000392,...,0.000392,0.000392,0.000392,0.000392,0.000392,0.000392,0.464354,0.000392,0.000392,0.000392
2018.1.01077.S,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,...,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.376111,0.000194,0.000194
2018.1.00437.S,0.000192,0.000192,0.000192,0.127172,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,...,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192
2021.1.00637.S,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,...,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222
2012.1.00786.S,0.000171,0.000171,0.192147,0.000171,0.000171,0.000171,0.000171,0.740628,0.000171,0.000171,...,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171


Take highest matching topic for each project

In [20]:
proj_topics['max_topic'] = proj_topics.apply(lambda x: x.argmax(), axis=1)

Create data frame with project id and max topic

In [21]:
proj_max_topic = proj_topics['max_topic'].to_frame()
proj_max_topic.head()

,max_topic
2016.1.01288.S,46
2018.1.01077.S,37
2018.1.00437.S,17
2021.1.00637.S,13
2012.1.00786.S,7


### Add `max_topic` to `measurements` frame to be able to group measurements by max topic

In [22]:
train_measurements = pd.merge(train_measurements, proj_max_topic, left_index=True, right_index=True)

In [23]:
proj_max_topic.value_counts().describe()

count     50.0000
mean      63.5600
std       64.2737
min        7.0000
25%       24.2500
50%       33.5000
75%       92.7500
max      293.0000
Name: count, dtype: float64

There are a few topics that match to a large number of documents. Perhaps we need a better topic model or to group documents by project_topic vector similarity.

Eyeball comparison of documents by max topic. This requires looking at the online explorer since printing out abstracts in here gets messy.

In [24]:
proj_max_topic[proj_max_topic.max_topic == 3].head()

,max_topic
2022.1.00793.S,3
2015.1.01260.S,3
2017.1.00167.S,3
2019.1.01443.T,3
2015.1.00032.S,3


### Generate test projects measurements
This will be useful for calculating hit rates to evaluate model performance.

**NOTE!!!**
You should not sort these, however tempting. We need to preserve the relationships of the entries to not lose measurement information.

In [25]:
test_proj_meas = test_measurements.loc[test_texts.index]
test_proj_meas = test_proj_meas.groupby(test_proj_meas.index)\
    .agg({
        'low_freq': lambda x: round(x, 4).tolist(),
        'high_freq': lambda x: round(x, 4).tolist(),
        'med_freq': lambda x: round(x, 4).tolist(),
        'diff_freq': lambda x: round(x, 4).tolist()
    })
test_proj_meas.head()

,low_freq,high_freq,med_freq,diff_freq
project_code,,,,
2011.0.00010.S,"[90.38, 90.7, 91.69, 92.89, 217.59, 218.67, 21...","[90.62, 90.93, 91.92, 93.12, 218.53, 219.6, 21...","[90.5, 90.815, 91.805, 93.005, 218.06, 219.135...","[0.24, 0.23, 0.23, 0.23, 0.94, 0.93, 0.94, 0.9..."
2011.0.00064.S,"[288.96, 290.79, 300.84, 302.71, 288.94, 290.7...","[290.84, 292.67, 302.71, 304.59, 290.82, 292.6...","[289.9, 291.73, 301.775, 303.65, 289.88, 291.7...","[1.88, 1.88, 1.87, 1.88, 1.88, 1.87, 1.88, 1.87]"
2011.0.00121.S,"[319.07, 320.48, 319.83, 319.36, 319.71, 316.59]","[320.94, 322.35, 321.71, 321.24, 321.58, 318.47]","[320.005, 321.415, 320.77, 320.3, 320.645, 317...","[1.87, 1.87, 1.88, 1.88, 1.87, 1.88]"
2011.0.00136.S,"[335.29, 335.98, 345.67, 346.47]","[335.52, 336.22, 345.91, 346.7]","[335.405, 336.1, 345.79, 346.585]","[0.23, 0.24, 0.24, 0.23]"
2011.0.00199.S,"[639.15, 645.41, 657.7, 661.7, 320.98, 322.12,...","[640.11, 646.37, 658.66, 662.66, 321.46, 322.6...","[639.63, 645.89, 658.18, 662.18, 321.22, 322.3...","[0.96, 0.96, 0.96, 0.96, 0.48, 0.48, 0.49, 0.48]"


### Generate train topic measurements
We will use these to engineer 'areas of interest' among topics using DBSCAN

**NOTE!!!**
You should not sort these, however tempting. We need to preserve the relationships of the entries to not lose measurement information.

In [26]:
train_topic_freqs = train_measurements.loc[train_texts.index]\
    .groupby('max_topic')\
    .agg({
        'low_freq': lambda x: round(x, 4).tolist(),
        'high_freq': lambda x: round(x, 4).tolist(),
        'med_freq': lambda x: round(x, 4).tolist(),
        'diff_freq': lambda x: round(x, 4).tolist(),
        'band': lambda x: x.astype('int64').tolist()
    })
train_topic_freqs.head()

,low_freq,high_freq,med_freq,diff_freq,band
max_topic,,,,,
0,"[257.18, 259.1, 260.42, 262.37, 260.84, 262.79...","[259.04, 260.97, 262.29, 264.24, 262.71, 264.6...","[258.11, 260.035, 261.355, 263.305, 261.775, 2...","[1.86, 1.87, 1.87, 1.87, 1.87, 1.87, 1.87, 1.8...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, ..."
1,"[342.03, 343.99, 354.03, 355.91, 478.54, 480.5...","[343.9, 345.86, 355.9, 357.79, 480.54, 482.54,...","[342.965, 344.925, 354.965, 356.85, 479.54, 48...","[1.87, 1.87, 1.87, 1.88, 2.0, 2.0, 2.0, 0.25, ...","[7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 6, 6, 6, 6, 6, ..."
2,"[330.25, 331.25, 342.52, 345.1, 215.39, 217.28...","[331.25, 333.25, 344.52, 346.1, 217.39, 219.28...","[330.75, 332.25, 343.52, 345.6, 216.39, 218.28...","[1.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, ...","[7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."
3,"[327.94, 329.94, 339.99, 342.24, 344.24, 354.2...","[329.82, 331.82, 341.86, 344.12, 346.12, 356.1...","[328.88, 330.88, 340.925, 343.18, 345.18, 355....","[1.88, 1.88, 1.87, 1.88, 1.88, 1.87, 1.87, 1.8...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 7, 7, 6, 6, 6, ..."
4,"[216.05, 217.07, 217.21, 218.19, 218.45, 219.5...","[216.17, 217.13, 217.27, 218.25, 218.5, 219.59...","[216.11, 217.1, 217.24, 218.22, 218.475, 219.5...","[0.12, 0.06, 0.06, 0.06, 0.05, 0.06, 0.06, 0.1...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, ..."


## Function ton Inspect of HDBSCAN on Specific Topics

In [27]:
inspect_topic = 25
inspect_topic_frame = pd.DataFrame.from_dict(dict(train_topic_freqs.loc[inspect_topic]))

hdb = HDBSCAN(min_cluster_size=5)\
    .fit(list(zip(inspect_topic_frame.med_freq)))

# Add labels to inspect_topic_frame
inspect_topic_frame['cluster_label'] = hdb.labels_
inspect_topic_frame = inspect_topic_frame.sort_values('cluster_label', ascending=False)
inspect_topic_frame.cluster_label = inspect_topic_frame.cluster_label.astype('str')

# Add noise binary column for plot symbol
inspect_topic_frame['noise'] = np.where(inspect_topic_frame.cluster_label == '-1', 1, 0)

# Cheat Some indexes to group clusters together alond index for y-axis
# This gives us a "meaningless" 2nd-dimension to an otherwise 1-D plot, but helps with visualization
inspect_topic_frame = inspect_topic_frame.reset_index().drop('index', axis=1)
inspect_topic_frame = inspect_topic_frame.reset_index()

# Noise and Signal
itf_noise = inspect_topic_frame.noise.sum()
itf_signal = inspect_topic_frame.shape[0] - itf_noise

# Set symbols for plot
# We set all points to be 'circle' using the px number 0, and then change noise to 'x'
symbols = list(np.zeros(np.unique(inspect_topic_frame.cluster_label).shape[0], 'int'))
symbols[-1] = 'x'


# Create plot
fig = px.scatter(inspect_topic_frame,
                 x='med_freq',
                 y='cluster_label',
                 color='cluster_label',
                 symbol='cluster_label',
                 symbol_sequence=symbols,
                 title=f"HDBSCAN Generated Clusters for Topic {inspect_topic} <br><sup>{itf_signal} Clustered Measurements with {itf_noise} Noise Measurements</sup>",
                 labels={
                     'med_freq':'Median Frequency (GHz)',
                     'index':'Index',
                     'cluster_label':'Cluster Label'
                 })
fig.update_traces(marker={'size': 15, 'opacity':0.5})
fig.show()

In [28]:
inspect_topic_frame[inspect_topic_frame.cluster_label.isin(['1', '2', '4'])].sort_values('med_freq')

,index,low_freq,high_freq,med_freq,diff_freq,band,cluster_label,noise
112,112,240.06,241.94,241.000,1.88,6,1,0
114,114,240.79,242.79,241.790,2.00,6,1,0
117,117,240.86,242.73,241.795,1.87,6,1,0
115,115,240.80,242.81,241.805,2.01,6,1,0
116,116,242.61,244.49,243.550,1.88,6,1,0
118,118,242.55,244.55,243.550,2.00,6,1,0
119,119,242.76,244.64,243.700,1.88,6,1,0
113,113,242.87,244.87,243.870,2.00,6,1,0
108,108,284.58,286.45,285.515,1.87,7,2,0
107,107,286.48,288.35,287.415,1.87,7,2,0


In [29]:
pd.DataFrame(train_topic_freqs.loc[0].med_freq,
             train_topic_freqs.loc[0].band)\
             .reset_index()

,index,0
0,6,258.110
1,6,260.035
2,6,261.355
3,6,263.305
4,6,261.775
...,...,...
120,6,247.295
121,6,249.065
122,6,261.235
123,6,263.005


## Cluster cleaning function
# NEEDS WORK HERE
Want to keep noise in cluster data frame and currently this relies on noise being removed

How do we handle >2 band overlaps?
    * Comparing to cluster density of raw data maybe helps?

In [30]:
# Code to check for clusters that span at least two bands
def cluster_cleaning(cluster_df, topic_df):
    new_rows = []
    bad_rows = []
    for clst in cluster_df.index:
        # HDBSCAN labels noise as -1 so we skip this row for cleaning
        if clst != -1:
            if cluster_df.loc[clst].band_min != cluster_df.loc[clst].band_max:
                olap_band_min = cluster_df.loc[clst].band_min.astype('int64')
                olap_band_max = cluster_df.loc[clst].band_max.astype('int64')
                olap_band_mode = cluster_df.loc[clst].band_mode.astype('int64')
                olap_min_freq = cluster_df.loc[clst].min_freq
                olap_max_freq = cluster_df.loc[clst].max_freq

                # Check to see that the cluster doesn't span more than two bands
                # If it does to this the cluster is far too large and clusters need to be tuned better
                if olap_band_max - olap_band_min > 1:
                    #raise ValueError('Cluster spans more than 2 bands. Re-parameterize clusters.')
                    print("WARNING: Generated cluster spans more than two bands.")
                
                # Otherwise, we split the cluster into two different clusters on the band boundaries
                new_row1_min_freq = olap_min_freq
                new_row1_max_freq = band_cutoffs[olap_band_max-1]
                new_row2_min_freq = band_cutoffs[olap_band_max-1]
                new_row2_max_freq = olap_max_freq

                # There are cases where the band transition in the data is wrong
                # Measurements in band 4 are misclassified to band 5 it seems if they are very close to the boundary between bands
                # If this is the case, we don't make new rows and simply set the band mode for the 'bad' row to the band with the higher mode
                if (new_row1_min_freq > new_row1_max_freq) or (new_row2_min_freq > new_row2_max_freq):
                    break

                reassign_measures = topic_df[topic_df.cluster_label == clst].med_freq\
                    .sort_values()\
                    .to_list()
                new_row1_measures = []
                new_row2_measures = []

                # Loop over reassign_measures and build lists for each new cluster
                for meas in reassign_measures:
                    if meas <= new_row2_min_freq:
                        new_row1_measures.append(meas)
                    else:
                        new_row2_measures.append(meas)
                
                # Generate column values for new rows (clusters)
                new_row1_count = len(new_row1_measures)
                new_row2_count = len(new_row2_measures)
                new_row1_mean = np.mean(new_row1_measures)
                new_row2_mean = np.mean(new_row2_measures)
                new_row1_band_min = olap_band_min
                new_row1_band_max = olap_band_min
                new_row2_band_min = olap_band_max
                new_row2_band_max = olap_band_max
                new_row1_band_mode = olap_band_mode.astype('int64')
                new_row2_band_mode = olap_band_mode.astype('int64')

                # Make new row lists to add to data frame
                new_row1 = [new_row1_mean,
                            new_row1_min_freq,
                            new_row1_max_freq,
                            new_row1_count,
                            new_row1_band_min,
                            new_row1_band_max,
                            new_row1_band_mode
                            ]
                
                new_row2 = [new_row2_mean,
                            new_row2_min_freq,
                            new_row2_max_freq,
                            new_row2_count,
                            new_row2_band_min,
                            new_row2_band_max,
                            new_row2_band_mode
                            ]
                
                # Add new rows (clusters) to list to ultimately alter cluster_df
                # We don't want to alter the data frame we're looping over in the loop
                new_rows.append(new_row1)
                cluster_df.loc[len(cluster_df.index)] = new_row1
                cluster_df.loc[len(cluster_df.index)] = new_row2
                bad_rows.append(clst)

    # Drop and add affected rows
    if len(bad_rows) != 0:
        for br in range(len(bad_rows)):
            cluster_df = cluster_df.drop(bad_rows[br], axis=0)
            
    # Set axis to maintain HDBSCAN noise is labeled -1
    cluster_df.index = (list(range(-1,cluster_df.shape[0]-1)))
    return cluster_df

### Loop over topics and find accuracy measurements

In [31]:
test_project_hits = 0                   # Hits for all projects if at least one measurement is matched
test_project_meas_hit_rate = []         # List of hit rates by project
topic_cluster_widths = []               # List of cluster widths by topic to ensure generated clusters are not too wide (list of lists)
total_num_clusters = 0                  # List of number of clusters for each topic
topic_cluster_stat_list = []            # List of dataframes with clusters by topic. Used to make a main topic-cluster data frame later

# Loop over topics
for tpc in set(proj_max_topic.max_topic.values):
    # Fit HDBSCAN for each topic
    # Note that these can be parameterized for each of the topics generated
    # Note one can give HDBSCAN a max_cluster_size parameter to ensure clusters do not grow too large
    db = HDBSCAN(min_cluster_size=5)\
        .fit(list(zip(train_topic_freqs.loc[tpc].med_freq)))
    
    # Get labels from HDBSCAN
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_points = len(list(labels))
    n_noise = list(labels).count(-1)

    # Stat callouts
    print(f'HDBSCAN Results for topic {tpc}')
    #print(f'Estimated number of clusters: {n_clusters}')
    print(f'Number of projects in topic: {proj_max_topic.loc[train_texts.index].query(f"max_topic == {tpc}").shape[0]}')
    print(f'Total number of measurements: {n_points}')
    print(f'Estimated number of noise measurements: {n_noise}')
    print(f'Noise proportion: {round(list(labels).count(-1)/labels.shape[0], 3)}')
    print(f'Signal proportion: {round(1-list(labels).count(-1)/labels.shape[0], 3)}')

    # Create data frame for measurements in this specific topic
    selected_topic = pd.DataFrame(train_topic_freqs.loc[tpc].med_freq,
                                  train_topic_freqs.loc[tpc].band)\
                                    .reset_index()
    selected_topic.columns = ['band', 'med_freq']
    selected_topic['cluster_label'] = labels

    # Generate topic_cluster data frame for this topic
    # Clusters are defined by the minimum and maximum median_freq for all labeled measurements
    topic_cluster = selected_topic.groupby('cluster_label').agg(
        mean_freq=('med_freq', 'mean'),
        min_freq=('med_freq', 'min'),
        max_freq=('med_freq', 'max'),
        count_freq=('med_freq', 'count'),
        band_min=('band', 'min'),
        band_max=('band', 'max'),
        band_mode=('band', 'mean')
    )

    # Break up clusters that span more than one band
    topic_cluster = cluster_cleaning(topic_cluster, selected_topic)
    topic_cluster = topic_cluster.sort_index()

    # Add width for cleaned clusters
    topic_cluster['width'] = topic_cluster.max_freq - topic_cluster.min_freq

    # Add topic index to topic_cluster
    topic_cluster = pd.concat({tpc: topic_cluster}, names=['topic'])
    topic_cluster.index.names = ['topic', 'cluster']

    # Append topic_cluster to topic_cluster_stats for analysis later
    topic_cluster_stat_list.append(topic_cluster)


    # Testing loop
    # Loop over generated clusters and print cluster stats
    # Initialize list of cluster widths
    # Code to check for clusters that span at least two bands
    cluster_widths = []

    # If after dropping noise there are still clusters, print the stats
    # This if statement is to avoid errors if topics only have noise and no clusters
    if topic_cluster.shape[0] != 0:
        for clst in topic_cluster.index:
            min_freq, max_freq = topic_cluster.loc[clst].min_freq, topic_cluster.loc[clst].max_freq
            cluster_widths.append(max_freq - min_freq)
            total_num_clusters += 1
            if (topic_cluster.loc[clst].band_min != topic_cluster.loc[clst].band_max):
                print("BAND OVERLAP")
        print('')
        print(f'Topic {tpc} cluster width stats:')
        print(np.round(pd.Series(cluster_widths).describe(), 4))
    else: print('No clusters for this topic')

    # Print cluster data frame with relevant columns for tuning
    print('')
    print(f'Cluster data frame for topic {tpc}')
    with pd.option_context('display.max_rows', None):
        print(topic_cluster[['min_freq', 'max_freq', 'count_freq', 'band_mode', 'width']]\
          .sort_values(['width', 'min_freq'], ascending=False))

    # Get a list of test project codes
    tps = proj_max_topic.loc[test_texts.index].query(f'max_topic == {tpc}')

    #Begin test projects
    print('')
    # print('Begin tests')

    # Loop over test projects
    for tp in tps.index:
        tp_hr = 0   # Hit rate for this specific project
        # Loop over measurements in test project
        for meas in test_proj_meas.loc[tp].med_freq:
            # Loop over clusters in topic
            # Note we have a multi index so we want to get to the 'cluster' index, being level 1
            for clust in topic_cluster.index.get_level_values(level=1):
                # Skip noise
                if clust != -1:
                    lower_bound = round(topic_cluster.loc[tpc, clust].min_freq, 3)
                    upper_bound = round(topic_cluster.loc[tpc, clust].max_freq, 3)
                    if ((meas >= lower_bound) and (meas <= upper_bound)):
                        tp_hr += 1
                        break
        test_project_meas_hit_rate.append(round(tp_hr/len(list(test_proj_meas.loc[tp].med_freq)), 3))
        #Print some stats
        print(f'Number of measurements: {len(test_proj_meas.loc[tp].med_freq)}')
        print(f'Hits: {tp_hr}')
        print(f'Hit rate: {round(tp_hr/len(list(test_proj_meas.loc[tp].med_freq)), 3)}')
        print('')

        # Increment test_project_hits if at least one measurement in the project matched
        if (tp_hr > 0):
            test_project_hits +=1
    print('=========================================\n')

print(f'Total number of clusters across topics: {total_num_clusters}')
print(f'Number of test projects with at least one measurement match: {test_project_hits}')
print(f'Ratio of test project hits to number of test projects: {round(test_project_hits/test_texts.shape[0], 4)}')
print(f'Average hit rate per project: {round(sum(test_project_meas_hit_rate)/test_texts.shape[0], 4)}')

HDBSCAN Results for topic 0
Number of projects in topic: 17
Total number of measurements: 125
Estimated number of noise measurements: 5
Noise proportion: 0.04
Signal proportion: 0.96
BAND OVERLAP

Topic 0 cluster width stats:
count     12.0000
mean      22.4333
std       43.6986
min        2.7400
25%        3.8688
50%        9.1325
75%       15.4650
max      159.3400
dtype: float64

Cluster data frame for topic 0
               min_freq  max_freq  count_freq  band_mode    width
topic cluster                                                    
0     -1         86.845   246.185         5.0        4.6  159.340
       10       158.000   182.000         6.0        4.0   24.000
       0        329.325   351.930        10.0        7.0   22.605
       9        144.915   158.000         2.0        4.0   13.085
       8         87.925    99.105        10.0        3.0   11.180
       5        258.110   269.140        25.0        6.0   11.030
       6        247.295   254.530         9.0        6.

### Build full topic-cluster data frame
This data frame holds all of the cluster info for each of the generated topics

In [32]:
topic_cluster_stats = pd.concat(topic_cluster_stat_list)

In [33]:
topic_cluster_stats.sample(10)

,,mean_freq,min_freq,max_freq,count_freq,band_min,band_max,band_mode,width
topic,cluster,,,,,,,,
12,47,97.905000,97.900,97.910,6.0,3.0,3.0,3.0,0.010
9,24,233.032273,232.945,233.255,11.0,6.0,6.0,6.0,0.310
14,4,290.414375,290.035,290.685,8.0,7.0,7.0,7.0,0.650
8,13,233.144167,232.975,233.495,6.0,6.0,6.0,6.0,0.520
16,26,254.565625,253.520,256.255,8.0,6.0,6.0,6.0,2.735
13,6,115.210000,115.160,115.270,7.0,3.0,3.0,3.0,0.110
49,104,243.669167,243.190,244.325,12.0,6.0,6.0,6.0,1.135
46,9,313.432727,303.495,319.920,11.0,7.0,7.0,7.0,16.425
48,7,320.212000,318.265,322.960,5.0,7.0,7.0,7.0,4.695


## Inspect topic-clusters

In [34]:
topic_cluster_stats.describe()

,mean_freq,min_freq,max_freq,count_freq,band_min,band_max,band_mode,width
count,1546.000000,1546.000000,1546.000000,1546.000000,1546.000000,1546.000000,1546.000000,1546.000000
mean,231.613453,225.606028,240.349143,11.408797,5.347348,5.483182,5.411664,14.743114
std,119.468344,119.766691,130.682048,14.614964,1.721833,1.736355,1.681171,68.828140
min,39.588750,36.080000,43.105000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,114.727125,111.972500,115.175000,6.000000,3.000000,3.000000,3.000000,0.236250
50%,227.484615,223.465000,230.355000,9.000000,6.000000,6.000000,6.000000,0.882500
75%,290.436370,277.983750,315.175000,12.000000,7.000000,7.000000,7.000000,3.583750
max,890.962188,878.230000,906.795000,277.000000,10.000000,10.000000,10.000000,750.670000


In [35]:
topic_cluster_stats.width.describe()

count    1546.000000
mean       14.743114
std        68.828140
min         0.000000
25%         0.236250
50%         0.882500
75%         3.583750
max       750.670000
Name: width, dtype: float64

In [36]:
topic_cluster_stats.query('width > 10 and cluster != -1')\
    .sort_values(['count_freq', 'width'], ascending=False)\
    .head(50)


,,mean_freq,min_freq,max_freq,count_freq,band_min,band_max,band_mode,width
topic,cluster,,,,,,,,
17,0,144.955000,128.640,161.730,73.0,4.0,4.0,4.0,33.090
18,1,342.669255,305.365,372.865,47.0,7.0,7.0,7.0,67.500
5,5,255.655111,248.805,263.955,45.0,6.0,6.0,6.0,15.150
12,2,141.371447,126.035,158.505,38.0,4.0,4.0,4.0,32.470
10,97,143.925968,127.505,158.000,31.0,4.0,4.0,4.0,30.495
15,3,142.426034,128.315,158.265,29.0,4.0,4.0,4.0,29.950
25,7,112.277586,105.285,115.290,29.0,3.0,3.0,3.0,10.005
49,2,671.203214,648.000,688.520,28.0,9.0,9.0,9.0,40.520
7,0,337.877778,310.995,356.730,27.0,7.0,7.0,7.0,45.735


In [37]:
topic_cluster_stats.query('band_max - band_min > 1 and cluster != -1')\
    .sort_values(['width', 'count_freq'], ascending=False)\
    .head(30)

,,mean_freq,min_freq,max_freq,count_freq,band_min,band_max,band_mode,width
topic,cluster,,,,,,,,


In [38]:
topic_cluster_stats.loc[11]

,mean_freq,min_freq,max_freq,count_freq,band_min,band_max,band_mode,width
cluster,,,,,,,,
-1,228.075000,227.075,229.075,2.0,6.0,6.0,6.0,2.000
0,356.548462,333.815,372.630,13.0,7.0,7.0,7.0,38.815
1,485.394687,469.435,497.195,16.0,8.0,8.0,8.0,27.760
2,113.541667,113.470,211.000,3.0,3.0,3.0,4.0,97.530
3,214.075000,211.000,215.075,2.0,6.0,6.0,4.0,4.075


In [39]:
import plotly.graph_objects as go
def plot_topic_clusters(tc_frame:pd.DataFrame, topic:int):
    figure = go.Figure()
    figure.add_trace(
        go.Bar(
            x=tc_frame.query(f'topic== {topic} and cluster != -1').mean_freq,
            y=tc_frame.query(f'topic== {topic} and cluster != -1').count_freq,
            #name=dict(color=tc_frame.query(f'topic== {topic} and cluster != -1').index.get_level_values(level=1)),
            width=tc_frame.query(f'topic== {topic} and cluster != -1').width.to_list()
            # hoverinfo=(
            #     tc_frame.query(f'topic== {topic} and cluster != -1').min_freq,
            #     tc_frame.query(f'topic== {topic} and cluster != -1').max_freq
            # )
        )
    )
    figure.update_layout(
    title=(f'Areas of Interest for Topic {topic}'),
    xaxis_title='Frequency (GHz)',
    yaxis_title='Count of Measurements',
    )
    figure.show()

In [40]:
plot_topic_clusters(topic_cluster_stats, 25)

## Looping in band prediction assessments

# NEEDS TO BE SYNCED WITH ABOVE TRAIN/TEST CODE
Taking the hit rates above as the upper limit, we see how reducing our recommendations to the 'areas of interest' from the top 2 predicted bands affects our hit rate

In [41]:
test_project_hits = 0               # Hits for all projects if at least one measurement is matched
test_project_meas_hit_rate = []     # List of hit rates by project
topic_cluster_widths = []           # List of cluster widths by topic to ensure generated clusters are not too wide (list of lists)
total_num_clusters = 0              # List of number of clusters for each topic

# Loop over topics
for tpc in set(proj_max_topic.max_topic.values):
    kaleigh_test = []
    # DBSCAN with parameters from topic parameter data frame
    # BASIC COMPARISON PARAMETERIZATION: eps=0.5, min_samples=2
    # db = DBSCAN(eps=0.25, min_samples=2)\
    #     .fit(list(zip(train_topic_freqs.loc[tpc].med_freq)))
    # db = DBSCAN(eps=params_frame.loc[tpc].eps, min_samples=2)\
    # .fit(list(zip(train_topic_freqs.loc[tpc].med_freq)))
    db = HDBSCAN(max_cluster_size=200, min_cluster_size=5)\
        .fit(list(zip(train_topic_freqs.loc[tpc].med_freq)))
    
    # Get labels from DBSCAN
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_points = len(list(labels))
    n_noise = list(labels).count(-1)

    # Stat callouts
    print(f'HDBSCAN Results for topic {tpc}')
    #print(f'Estimated number of clusters: {n_clusters}')
    print(f'Number of projects in topic: {proj_max_topic.loc[train_texts.index].query(f"max_topic == {tpc}").shape[0]}')
    print(f'Total number of measurements: {n_points}')
    print(f'Estimated number of noise measurements: {n_noise}')
    print(f'Noise percentage: {round(list(labels).count(-1)/labels.shape[0], 3)}')
    print(f'Signal to noise ratio: {round(1-list(labels).count(-1)/labels.shape[0], 3)}')

    # Create data frame for measurements in this specific topic
    selected_topic = pd.DataFrame(train_topic_freqs.loc[tpc].med_freq,
                                  train_topic_freqs.loc[tpc].band)\
    .reset_index()
    selected_topic.columns = ['band', 'med_freq']
    selected_topic['cluster_label'] = labels

    # Take mean of diff_freq and med_freq to generate areas of interest    
    topic_cluster = selected_topic.groupby('cluster_label').agg(
        mean_freq=('med_freq', 'mean'),
        min_freq=('med_freq', 'min'),
        max_freq=('med_freq', 'max'),
        count_freq=('med_freq', 'count'),
        band_min=('band', 'min'),
        band_max=('band', 'max'),
        band_mode=('band', 'mean')
    )
    # We will leverage the band mode, but this is not working in .agg
    # Instead we take the mean of band and round to the nearest value to approximate the mode
    topic_cluster.band_mode = round(topic_cluster.band_mode, 0).astype('int')

    # Check to see if there is noise from clustering
    # If so, drop noise cluster as we do not want to count hits there
    if (-1 in topic_cluster.index):
        topic_cluster = topic_cluster.drop(-1, axis=0) # Drop noise (label -1)
    topic_cluster = topic_cluster.sort_index()

    # Break up clusters that span more than one band
    topic_cluster = cluster_cleaning(topic_cluster, selected_topic)
    topic_cluster = topic_cluster.sort_index()

    # Loop over generated clusters and print cluster stats
    # Initialize list of cluster widths
    # Code to check for clusters that span at least two bands
    new_rows = []
    bad_rows = []
    cluster_widths = []

    # If after dropping noise there are still clusters, print the stats
    # This if statement is to avoid errors if topics only have noise and no clusters
    if topic_cluster.shape[0] != 0:
        for clst in topic_cluster.index:
            min_freq, max_freq = topic_cluster.loc[clst].min_freq, topic_cluster.loc[clst].max_freq
            cluster_widths.append(max_freq - min_freq)
            total_num_clusters += 1
            if (topic_cluster.loc[clst].band_min != topic_cluster.loc[clst].band_max):
                print("BAND OVERLAP")
        print('')
        print(f'Topic {tpc} cluster width stats:')
        print(np.round(pd.Series(cluster_widths).describe(), 4))
    else: print('No clusters for this topic')

    # Print topic cluster dataframe for HDBSCAN tuning
    print('')
    print(topic_cluster.sort_values('count_freq', ascending=False))

    # Get a list of test project codes
    tps = proj_max_topic.loc[test_texts.index].query(f'max_topic == {tpc}')

    # Check to see if there are any test projects assigned to this topic
    # ADD IF, ELSE STATEMENT HERE, ATTACH ELSE TO FOLLOWING CODE

    #Begin test projects
    print('')
    # print('Begin tests')

    # Loop over test projects
    for tp in tps.index:
        tp_hr = 0   # Hit rate for this specific project
        #print(f'Test project {tp}:')

        # Subset `topic_cluster` to be the top two predicted bands for each project
        topic_cluster_subset = topic_cluster[topic_cluster.band_mode.isin(literal_eval(band_predictions.loc[tp].band_predictions)[-2:])]
        if topic_cluster.shape[0] != 0:
            print(f'Ratio of recommended clusters to total clusters: {topic_cluster_subset.shape[0]/topic_cluster.shape[0]}')
        else: print(f'No clusters for topic {tpc}')

        # Loop over measurements in test project
        for meas in test_proj_meas.loc[tp].med_freq:
            # Loop over clusters in topic
            for clust in topic_cluster_subset.index.values:
                lower_bound = round(topic_cluster_subset.loc[clust].min_freq, 3)
                upper_bound = round(topic_cluster_subset.loc[clust].max_freq, 3)
                if ((meas >= lower_bound) and (meas <= upper_bound)):
                    tp_hr += 1
                    break
        test_project_meas_hit_rate.append(round(tp_hr/len(list(test_proj_meas.loc[tp].med_freq)), 3))
        #Print some stats
        # print(f'Number of measurements: {len(test_proj_meas.loc[tp].med_freq)}')
        # print(f'Hits: {tp_hr}')
        # print(f'Hit rate: {round(tp_hr/len(list(test_proj_meas.loc[tp].med_freq)), 3)}')
        # print('')

        # Increment test_project_hits if at least one measurement in the project matched
        if (tp_hr > 0):
            test_project_hits +=1
    print('=========================================\n')

print(f'Total number of clusters across topics: {total_num_clusters}')
print(f'Number of test projects with at least one measurement match: {test_project_hits}')
print(f'Ratio of test project hits to number of test projects: {round(test_project_hits/test_texts.shape[0], 4)}')
print(f'Average hit rate per project: {round(sum(test_project_meas_hit_rate)/test_texts.shape[0], 4)}')

HDBSCAN Results for topic 0
Number of projects in topic: 17
Total number of measurements: 125
Estimated number of noise measurements: 5
Noise percentage: 0.04
Signal to noise ratio: 0.96

Topic 0 cluster width stats:
count    11.0000
mean      9.9873
std       7.4688
min       2.7400
25%       3.7575
50%       7.2350
75%      12.1325
max      24.0000
dtype: float64

     mean_freq  min_freq  max_freq  count_freq  band_min  band_max  band_mode
 4  263.754800   258.110   269.140        25.0       6.0       6.0        6.0
 3  218.598333   216.860   220.395        15.0       6.0       6.0        6.0
 6  109.520385   108.155   110.895        13.0       3.0       3.0        3.0
 2  231.342727   230.470   233.810        11.0       6.0       6.0        6.0
-1  341.727000   329.325   351.930        10.0       7.0       7.0        7.0
 1  463.914000   459.095   466.225        10.0       8.0       8.0        8.0
 7   94.646500    87.925    99.105        10.0       3.0       3.0        3.0
 0  453


Topic 8 cluster width stats:
count    18.0000
mean      7.1469
std       8.7681
min       0.0100
25%       0.4175
50%       3.4650
75%      11.2050
max      25.2200
dtype: float64

      mean_freq  min_freq  max_freq  count_freq  band_min  band_max  band_mode
 6   345.817059   342.875   351.740          17         7         7          7
 2    97.625625    85.025   110.245          16         3         3          3
 9   243.053000   239.985   245.590          10         6         6          6
 3   332.866000   330.575   336.475          10         7         7          7
 15  220.439444   220.380   220.610           9         6         6          6
 14  218.270000   217.435   218.760           9         6         6          6
 5   357.243889   354.405   360.165           9         7         7          7
-1   885.891250   878.230   897.500           8        10        10         10
 10  235.955000   235.355   236.385           8         6         6          6
 16  219.701875   219.550   